on homework 5, analytical solution of y(x)=sinx, and dy/dx=cosx; with period of 2pi

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

Creating a notebook to perform runge-kutta integraton for multiple coupled variables

## Define our coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    #Set derivatives
    #y is an array, 2D array
    #1 dimension is x, other one is 1D with y and x elements
    #our eqn is d^2y/dx^2 = -y
    #so we can write dydz=z, dzdx=-y
    #we will set y = y[0], z = y[1]
    #declare an array
    
    y_derivs = np.zeros(2)
    
    #set dydx=z
    y_derivs[0] = y[1]
    
    #set dxdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have an array
    return y_derivs

## Define the 4th order Runge Kutta method

In [ ]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    #nv is number of variables
    #? is wild card, it can be any digit from 0-9
    #each k give us derivatives estimates of different fxns that were trying to integrate
    
    #define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    #this is an array
    return yipo

## Define adaptive step size driver for RK4
This adjusts step size, if step is too big, and error estimate is 
bigger than tolerance, must retake and correct by taking a smaller step, because the step taken was too big before based on the smoothness of fxn at that point. it adjusts the error and corrects the step size. some steps will be smaller, and some will be bigger depending on smoothness of fxn. dont know how many steps are being taken because the size of steps vary. must align the last and first steps with the boundary conditions at those points. dont want to overshoot

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    #define safety scale, tell us how much our step is gonna change by
    SAFETY      = 0.9
    H_NEW_FAC   = 2.0
    # HNF max fator by which were taking a bigger step
    
    #set max number of iterations becasue were doing a while loop
    imax = 1000
    
    #set iteration variable
    i = 0
    
    #create an error, delta is an array, size of nv, fill it by number that is twice our tolerance
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate error by taking 1 step size of h vs. 2 steps of size h/2
        y_2  = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1  = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_i,nv,0.5*h_step)
        
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        
        #if error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #Check iteration
        if(i>=imax):
            print("Too many iterations in rk4_mv_as()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i+=1
        
    #next time try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step * H_NEW_FAC)
    
    #return the answer, a new step, an the step we actually took
    return y_2, h_new, h_step

## Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    #dydx is the derivative wrt x
    #a is lower bound, b is upper bound
    #y_a are the boundary conditions array that contains values from 0-1
    #tol is the tolerance for integrating y
    
    #define starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set max number of iterations
    imax = 10000
    
    #set iteration variable
    i = 0
    
    #set number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    #set initial conditions, makes np array, y is 2D array, with 2 indices to y
    #1st is along x direction, 2nd has 2 values y and z
    #2 arrays side by side, 1 for y and 1 for z
    x = np.full(l,a)
    y = np.full((1,nv),y_a)
    
    #set flag/initial conditions
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update step
        h = h_new
        
        #prevent overshoot
        if(xi+h_step>b):
            
            #take smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>imax):
            
            print("Max number of iterations reached")
            raise StopIteration("Iteration number = ",i)
            
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\tb=%9.8f" % (i,xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
    
    #return the answer
    return x,y